# Recomendação de Filmes

Vamos utilizar o surprise para resolver um problema de recomendação de filmes, baseado em notas de avaliação dos usuário. O dataset utlizado foi o [MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/), com 100k avaliações de de 600 usários em 9k filmes.

A recomendação de filmes é algo comum encontrado nas plataformas de streaming, mas a recomendação no geral está presente em muitas outras aplicações.

## 1. Importando as Bibliotecas

Vamos começar instalando a biblioteca surprise.

In [227]:
!pip install surprise

Agora podemos importar todos os pacotes q vamos utilizar.

In [228]:
import pandas as pd
import numpy as np

from surprise import Reader, Dataset
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVDpp
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy


## 2. Carregando Dados

Vamos carregar todas as nossas base de dados.

In [229]:
links = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/links.csv')
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [230]:
movies = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/movies.csv')
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [231]:
ratings = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/ratings.csv')
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [232]:
tags = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/tags.csv')
tags.head(2)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


## 3. EDA

Agora vamos dar uma analisada nos dados que temos. Sigamos com a nossa Análise Exploratória dos Dados.

In [233]:
# Juntando as informações de filmes e avaliações
filmes = ratings.join(movies.set_index('movieId'), on='movieId')

In [234]:
print('Tamanho da Base de Filmes: ', filmes.shape)

Tamanho da Base de Filmes:  (100836, 6)


In [235]:
filmes.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [236]:
# Números dos datasets
print('Quantidade de Filmes Avaliados: ',
filmes['movieId'].value_counts().shape[0])

print('Quantidade de Usuários Avaliando: ',
filmes['userId'].value_counts().shape[0])

print('Quantidade de Avaliações: ',
ratings.shape[0])

Quantidade de Filmes Avaliados:  9724
Quantidade de Usuários Avaliando:  610
Quantidade de Avaliações:  100836


In [237]:
# Quantidade de Avaliações TOP5 Filmes
filmes['title'].value_counts().head()

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: title, dtype: int64

In [238]:
# Quantidade de Avaliações LOW5 Filmes
filmes['title'].value_counts().tail()

Abduction (2011)                             1
A Most Wanted Man (2014)                     1
Day for Night (La Nuit Américaine) (1973)    1
Too Big to Fail (2011)                       1
Obsession (1976)                             1
Name: title, dtype: int64

In [239]:
# Quantidade de Avaliações TOP5 Usuários
filmes['userId'].value_counts().head()

414    2698
599    2478
474    2108
448    1864
274    1346
Name: userId, dtype: int64

In [240]:
# Quantidade de Avaliações LOW5 Usuários
filmes['userId'].value_counts().tail()

406    20
595    20
569    20
431    20
442    20
Name: userId, dtype: int64

In [241]:
# Avaliações do usuário 414
filmes.query('userId == 414').head()

,userId,movieId,rating,timestamp,title,genres
62294,414,1,4.0,961438127,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
62295,414,2,3.0,961594981,Jumanji (1995),Adventure|Children|Fantasy
62296,414,3,4.0,961439278,Grumpier Old Men (1995),Comedy|Romance
62297,414,5,2.0,961437647,Father of the Bride Part II (1995),Comedy
62298,414,6,3.0,961515642,Heat (1995),Action|Crime|Thriller


In [242]:
# Avaliações do usuário 414
filmes.query('userId == 414 and movieId == 1')

,userId,movieId,rating,timestamp,title,genres
62294,414,1,4.0,961438127,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


## 4. Modelo

In [243]:
# Configuração para treinamento
reader = Reader(rating_scale=(0,5))
# Seleção das variáveis para o modelo
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Divisão dos dados de treino e teste
trainset, testset = train_test_split(data, test_size=.25, random_state=42)

In [244]:
# Configurações das medidas de similaridade
sim_options = { 'name': 'pearson_baseline', 'user_based': True }

### 4.1 Treinamento do Modelo


Um algoritmo básico que leva em consideração uma classificação de linha de base.

In [245]:
knn = KNNBaseline(k=33, sim_options=sim_options)
knn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Um algoritmo simples, mas preciso.

In [246]:
slo = SlopeOne()
slo.fit(trainset)

Um algoritmo baseado em co-clustering. 
Os clusters são atribuídos usando um método de otimização direto, muito parecido com o k-means.


In [247]:
co = CoClustering(n_epochs=10, verbose=True, random_state=42)
co.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


### 4.2 Avaliação do Modelo

Para avaliar os modelos utilizamos o erro quadrático médio das previsões.

In [248]:
predictions_knn = knn.test(testset)
accuracy.rmse(predictions_knn)

RMSE: 0.8888


0.8887723564425568

In [249]:
predictions_co = co.test(testset)
accuracy.rmse(predictions_co)

RMSE: 0.9534


0.953389277913654

In [250]:
predictions_slo = slo.test(testset)
accuracy.rmse(predictions_slo)

RMSE: 0.9134


0.9134291817129228

### 4.3 Predição do Modelo

Vamos ver os parâmetros que são atribuidos as predições.

Parameters:	
* uid – The (raw) user id. See this note.
* iid – The (raw) item id. See this note.
* r_ui (float) – The true rating rui.
* est (float) – The estimated rating r^ui.
* details (dict) – Stores additional details about the prediction that might be useful for later analysis.

In [251]:
# Predição com os dados de teste
predictions_co[:5]

[Prediction(uid=50, iid=4282, r_ui=3.5, est=3.502175149086966, details={'was_impossible': False}),
 Prediction(uid=603, iid=2993, r_ui=3.0, est=3.656574203039672, details={'was_impossible': False}),
 Prediction(uid=140, iid=11, r_ui=4.0, est=3.7622324437457833, details={'was_impossible': False}),
 Prediction(uid=262, iid=497, r_ui=4.0, est=3.7268674304003677, details={'was_impossible': False}),
 Prediction(uid=492, iid=1363, r_ui=4.0, est=3.6653260183725465, details={'was_impossible': False})]

A função trainset utiliza um id interno para serem utilizados no modelo, eles podem ser obtidos através da função to_inner, e a função to_raw retorna os valores originais.

In [252]:
# uid=247, iid=364 Conversão para o id interno
print('Conversão Interna')
print('User:', co.trainset.to_inner_uid(414))
print('Movie:', co.trainset.to_inner_iid(1))

# uid=247, iid=364 Conversão para o id externo
print('Conversão Externa')
print('User:', co.trainset.to_raw_uid( co.trainset.to_inner_uid(414) ))
print('Movie:', co.trainset.to_raw_iid( co.trainset.to_inner_iid(1) ))

Conversão Interna
User: 21
Movie: 287
Conversão Externa
User: 414
Movie: 1


#### 4.3.1. Função Análise de Recomendação

Passando um ID de usuário e um ID de um filme, a função vai retornar a Estimativa de Avaliação do usuário para o filme informado. Caso seja um filme já avaliado pelo usuário retorna também a avaliação real.

In [253]:
def recomenda_filme(userId,movieId):
  # ID do usuário para predição
  uid = userId
  # ID do filme para predição
  iid = movieId 
  nome_filme = movies.query('movieId == @movieId')['title'].values[0]
  print('Filme:', nome_filme)
  print('Usuário:', userId)
  if filmes.query('userId == @userId and movieId == @movieId')['title'].values.size == 0:
    print('Usuário não avaliou o filme!')
  else:
    nota_filme = ratings.query('userId == @userId and movieId == @movieId')['rating'].values[0]
    print('Avaliação do usuário:', nota_filme)
  # Predição baseada no melhor modelo
  print('Estimativa de Avaliação[0-5]:', round(co.predict(co.trainset.to_raw_uid(uid), co.trainset.to_raw_iid(iid))[3], 2))

In [254]:
recomenda_filme(406,1)

Filme: Toy Story (1995)
Usuário: 406
Usuário não avaliou o filme!
Estimativa de Avaliação[0-5]: 4.17


In [255]:
recomenda_filme(414,1) 

Filme: Toy Story (1995)
Usuário: 414
Avaliação do usuário: 4.0
Estimativa de Avaliação[0-5]: 4.27


#### 4.3.2. Função TOP Recomendações

Passando um ID de usuário e a quantidade de recomendações, a função informa em ordem de recomendação os filmes para o usuário informado.

In [256]:
def top_n(userId,n):
  # Selecionando apenas os filmes do treinamento
  lista_filmes_treino = []
  for x in trainset.all_items():
    lista_filmes_treino.append(trainset.to_raw_iid(x))
  # Selecionando os filmes do treinamento que o usuário não avaliou
  filmes_user = ratings.query('userId == @userId')['movieId'].values
  filmes_user_nao = movies.query('movieId not in @filmes_user')
  filmes_user_nao = filmes_user_nao.query('movieId in @lista_filmes_treino')['movieId'].values
  # Criando um ranking para o usuário para os filmes não avaliados
  ranking=[]
  for movieId in filmes_user_nao:
    ranking.append((movieId, co.predict(co.trainset.to_inner_uid(userId), co.trainset.to_inner_iid(movieId))[3]))
  # Ordenando os TOP filmes avaliados
  ranking.sort(key=lambda x: x[1], reverse=True)
  # Selecionando os Ids dos filmes
  x,_ = zip(*ranking[:n])
  # Listando os nomes dos filmes em ordem de recomendação
  return movies.query('movieId in @x')['title'].copy().reset_index(drop=True)

In [257]:
top_n(414,5)

0                     Missing (1982)
1                    Paradise (1982)
2                Little Women (1949)
3               Warriors, The (1979)
4    Silence, The (Tystnaden) (1963)
Name: title, dtype: object

In [258]:
top_n(406,5)

0    When Night Is Falling (1995)
1         Mighty Aphrodite (1995)
2          Misérables, Les (1995)
3             Crimson Tide (1995)
4               Waterworld (1995)
Name: title, dtype: object